In [51]:
import numpy as np
import pandas as pd

# from pyspark.sql import SparkSession
# from pyspark.sql import functions as f
# from pyspark.sql.types import *

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, LSTM, Dropout

In [14]:
data = pd.read_csv("../data/scaled_data.csv")

In [15]:
data

,route,day,location,incident,min_delay,min_gap,direction,vehicle,day_month,month,hour,min,day_indexed,location_indexed,incident_indexed,direction_indexed,features,scaled_features
0,69,Sunday,warden_station,Security,22,44,S,8407,1,1,2,34,6.0,7.0,2.0,1.0,"[69.0,44.0,8407.0,1.0,1.0,2.0,34.0,6.0,7.0,2.0...","[0.007139857202855943,0.48888888888888893,0.09..."
1,35,Sunday,jane_station,Cleaning - Unsanitary,30,60,N,1051,1,1,3,6,6.0,15.0,8.0,0.0,"[35.0,60.0,1051.0,1.0,1.0,3.0,6.0,6.0,15.0,8.0...","[0.0035699286014279717,0.6666666666666667,0.01..."
2,900,Sunday,kipling_station,Security,17,17,unknown,3334,1,1,3,14,6.0,1.0,2.0,4.0,"[900.0,17.0,3334.0,1.0,1.0,3.0,14.0,6.0,1.0,2....","[0.09439311213775725,0.18888888888888888,0.036..."
3,85,Sunday,meadowale_loop,Security,1,1,unknown,1559,1,1,3,43,6.0,3248.0,2.0,4.0,"[85.0,1.0,1559.0,1.0,1.0,3.0,43.0,6.0,3248.0,2...","[0.00881982360352793,0.011111111111111112,0.01..."
4,40,Sunday,kipling_station,Emergency Services,0,0,unknown,0,1,1,3,47,6.0,1.0,5.0,4.0,"[40.0,0.0,0.0,1.0,1.0,3.0,47.0,6.0,1.0,5.0,4.0]","[0.0040949181016379675,0.0,0.0,0.0,0.0,0.13043..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53248,41,Sunday,eglinton/keele,Operations - Operator,28,53,N,1133,31,12,1,26,6.0,20.0,1.0,0.0,"[41.0,53.0,1133.0,31.0,12.0,1.0,26.0,6.0,20.0,...","[0.004199916001679967,0.5888888888888889,0.012..."
53249,94,Sunday,castle_frank_station,Emergency Services,10,20,unknown,8415,31,12,1,31,6.0,30.0,5.0,4.0,"[94.0,20.0,8415.0,31.0,12.0,1.0,31.0,6.0,30.0,...","[0.009764804703905922,0.22222222222222224,0.09..."
53250,63,Sunday,oakwood/vaughan,Diversion,33,43,N,7010,31,12,1,40,6.0,140.0,7.0,0.0,"[63.0,43.0,7010.0,31.0,12.0,1.0,40.0,6.0,140.0...","[0.006509869802603948,0.4777777777777778,0.077..."
53251,34,Sunday,eglinton_station,Emergency Services,0,0,N,8761,31,12,1,54,6.0,4.0,5.0,0.0,"[34.0,0.0,8761.0,31.0,12.0,1.0,54.0,6.0,4.0,5....","[0.0034649307013859725,0.0,0.09624934083318684..."


In [35]:
y = data["min_delay"]
y

0        22
1        30
2        17
3         1
4         0
         ..
53248    28
53249    10
53250    33
53251     0
53252    25
Name: min_delay, Length: 53253, dtype: int64

In [36]:
X = data[['route', 'min_gap', 'vehicle', 'day_month', 'month', 'hour', 'min', 'day_indexed', 'location_indexed', 'incident_indexed', 'direction_indexed']]
X

,route,min_gap,vehicle,day_month,month,hour,min,day_indexed,location_indexed,incident_indexed,direction_indexed
0,69,44,8407,1,1,2,34,6.0,7.0,2.0,1.0
1,35,60,1051,1,1,3,6,6.0,15.0,8.0,0.0
2,900,17,3334,1,1,3,14,6.0,1.0,2.0,4.0
3,85,1,1559,1,1,3,43,6.0,3248.0,2.0,4.0
4,40,0,0,1,1,3,47,6.0,1.0,5.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...
53248,41,53,1133,31,12,1,26,6.0,20.0,1.0,0.0
53249,94,20,8415,31,12,1,31,6.0,30.0,5.0,4.0
53250,63,43,7010,31,12,1,40,6.0,140.0,7.0,0.0
53251,34,0,8761,31,12,1,54,6.0,4.0,5.0,0.0


In [44]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled

array([[-0.42564102,  1.10316612,  0.74408886, ..., -0.54039381,
        -0.12709941, -0.58662666],
       [-0.53556461,  2.09831555, -1.23600867, ..., -0.53688155,
         2.01187398, -1.30397101],
       [ 2.26102083, -0.57614855, -0.62146779, ..., -0.54302801,
        -0.12709941,  1.56540641],
       ...,
       [-0.4450393 ,  1.04096928,  0.36804261, ..., -0.48200243,
         1.65537842, -1.30397101],
       [-0.53879766, -1.63349482,  0.83937904, ..., -0.54171091,
         0.94238729, -1.30397101],
       [-0.51616633,  1.66293767,  0.44260582, ..., -0.46619724,
         0.58589172, -1.30397101]])

In [45]:
train_X, test_X, train_y, test_y = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [48]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

def evaluator(model):
    # Dự đoán trên tập kiểm tra
    predictions = model.predict(test_X)

    # Đánh giá bằng MSE
    mse = mean_squared_error(test_y, predictions)

    # Đánh giá bằng RMSE
    rmse = np.sqrt(mse)

    # Đánh giá bằng MAE
    mae = mean_absolute_error(test_y, predictions)

    # Đánh giá bằng R2
    r2 = r2_score(test_y, predictions)

    print("MSE:", mse)
    print("RMSE:", rmse)
    print("MAE:", mae)
    print("R-squared:", r2)


In [67]:
# Xây dựng mô hình CNN
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(train_X.shape[1], 1)))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

e:\HOC\KLTN\.venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [68]:
model.fit(train_X, train_y, epochs=100, batch_size=32, validation_data=(test_X, test_y), verbose=2)


Epoch 1/100
1332/1332 - 5s - 4ms/step - loss: 33.3809 - val_loss: 19.0440
Epoch 2/100
1332/1332 - 4s - 3ms/step - loss: 17.2502 - val_loss: 16.9429
Epoch 3/100
1332/1332 - 3s - 3ms/step - loss: 16.6523 - val_loss: 16.6563
Epoch 4/100
1332/1332 - 4s - 3ms/step - loss: 15.7622 - val_loss: 16.4593
Epoch 5/100
1332/1332 - 4s - 3ms/step - loss: 15.1395 - val_loss: 16.3134
Epoch 6/100
1332/1332 - 4s - 3ms/step - loss: 14.8009 - val_loss: 15.7243
Epoch 7/100
1332/1332 - 4s - 3ms/step - loss: 15.1414 - val_loss: 15.6929
Epoch 8/100
1332/1332 - 3s - 3ms/step - loss: 14.3506 - val_loss: 15.8993
Epoch 9/100
1332/1332 - 3s - 2ms/step - loss: 14.2951 - val_loss: 17.1041
Epoch 10/100
1332/1332 - 3s - 3ms/step - loss: 14.2532 - val_loss: 15.6167
Epoch 11/100
1332/1332 - 4s - 3ms/step - loss: 13.9859 - val_loss: 15.4976
Epoch 12/100
1332/1332 - 8s - 6ms/step - loss: 14.3306 - val_loss: 17.0138
Epoch 13/100
1332/1332 - 9s - 7ms/step - loss: 13.8892 - val_loss: 15.9614
Epoch 14/100
1332/1332 - 10s - 8ms

In [70]:
model.save("../model/CNN.h5")

In [71]:
evaluator(model)

333/333 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
MSE: 20.330270090861575
RMSE: 4.508910077930317
MAE: 1.7240779527911847
R-squared: 0.786525066654351


In [53]:
def init_model():
    regressor = Sequential()
    regressor.add(LSTM(units=50, return_sequences=True, input_shape=(train_X.shape[1], 1)))
    regressor.add(Dropout(0.2))
    regressor.add(LSTM(units=50, return_sequences=True))
    regressor.add(Dropout(0.2))
    regressor.add(LSTM(units=50, return_sequences=True))
    regressor.add(Dropout(0.2))
    regressor.add(LSTM(units=50))
    regressor.add(Dropout(0.2))
    regressor.add(Dense(units=1))
    regressor.compile(optimizer='adam', loss='mean_squared_error')
    return regressor

In [62]:
LSTM_model = init_model()
LSTM_model.fit(train_X, train_y, epochs=100, batch_size=32, validation_data=(test_X, test_y), verbose=2)

Epoch 1/100


e:\HOC\KLTN\.venv\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1332/1332 - 23s - 17ms/step - loss: 66.8692 - val_loss: 27.9082
Epoch 2/100
1332/1332 - 18s - 14ms/step - loss: 25.3422 - val_loss: 23.5046
Epoch 3/100
1332/1332 - 19s - 14ms/step - loss: 20.8872 - val_loss: 19.1200
Epoch 4/100
1332/1332 - 18s - 14ms/step - loss: 18.5742 - val_loss: 16.4060
Epoch 5/100
1332/1332 - 19s - 14ms/step - loss: 17.6067 - val_loss: 16.4444
Epoch 6/100
1332/1332 - 18s - 14ms/step - loss: 17.1502 - val_loss: 17.0472
Epoch 7/100
1332/1332 - 17s - 13ms/step - loss: 17.0304 - val_loss: 15.9713
Epoch 8/100
1332/1332 - 18s - 13ms/step - loss: 16.6749 - val_loss: 16.9086
Epoch 9/100
1332/1332 - 18s - 13ms/step - loss: 16.7754 - val_loss: 16.4002
Epoch 10/100
1332/1332 - 18s - 14ms/step - loss: 16.5332 - val_loss: 15.9384
Epoch 11/100
1332/1332 - 19s - 14ms/step - loss: 16.3246 - val_loss: 15.7169
Epoch 12/100
1332/1332 - 18s - 13ms/step - loss: 16.1893 - val_loss: 16.4564
Epoch 13/100
1332/1332 - 18s - 13ms/step - loss: 16.0843 - val_loss: 15.5968
Epoch 14/100
1332/13

In [63]:
evaluator(LSTM_model) 

333/333 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step
MSE: 14.429936076443909
RMSE: 3.7986755687270675
MAE: 1.3546046615262488
R-squared: 0.848480633639713


In [64]:
LSTM_model.save("../model/LSTM.h5")